In [242]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point

In [243]:
stations = pd.DataFrame.from_csv('stations.csv',index_col=None)
stations.head()

type(stations)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


pandas.core.frame.DataFrame

In [244]:
stations.head()

,Station,LATITUDE,LONGITUDE,ELEVATION
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8


In [245]:
#stations = stations[stations['Station'].str.startswith('US')]

In [246]:
stations.head()

,Station,LATITUDE,LONGITUDE,ELEVATION
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8


In [225]:
stations['Longitude'] = -1 * (stations['Longitude'] - 180)

In [247]:
stations.head(10)

,Station,LATITUDE,LONGITUDE,ELEVATION
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8
5,AEM00041218,24.2620,55.6090,264.9
6,AF000040930,35.3170,69.0170,3366.0
7,AFM00040938,34.2100,62.2280,977.2
8,AFM00040948,34.5660,69.2120,1791.3
9,AFM00040990,31.5000,65.8500,1010.0


In [248]:
#original long, lat
geom = stations.apply(lambda x : Point([x['LONGITUDE'],x['LATITUDE']]),axis=1)

In [249]:
stations = gpd.GeoDataFrame(stations, geometry=geom) #geom is a Series

In [250]:
stations.crs = {'init' :'epsg:4326'}

In [251]:
stations.head()

,Station,LATITUDE,LONGITUDE,ELEVATION,geometry
0,ACW00011604,17.1167,-61.7833,10.1,POINT (-61.7833 17.1167)
1,ACW00011647,17.1333,-61.7833,19.2,POINT (-61.7833 17.1333)
2,AE000041196,25.3330,55.5170,34.0,POINT (55.517 25.333)
3,AEM00041194,25.2550,55.3640,10.4,POINT (55.364 25.255)
4,AEM00041217,24.4330,54.6510,26.8,POINT (54.651 24.433)


In [252]:
geojson_file = 'final_ merged.geojson'

In [253]:
cities = gpd.read_file(geojson_file)[['NAME','geometry']]

#cities = gpd.read_file(geojson_file)[['NAME','geometry']]

In [254]:
cities.head()

,NAME,geometry
0,Valley Head,"(POLYGON ((-85.630563 34.555064, -85.626993 34..."
1,Sylvania,"(POLYGON ((-85.781384 34.565536, -85.776886 34..."
2,Shiloh,"(POLYGON ((-85.886044 34.462529, -85.885583999..."
3,Rainsville,"(POLYGON ((-85.895725 34.513797, -85.891408 34..."
4,Powell,"(POLYGON ((-85.869552 34.524696, -85.871782 34..."


In [268]:
stations_and_cities = gpd.sjoin(cities,stations,op='intersects')

In [269]:
stations_and_cities.head()

,NAME,geometry,index_right,Station,LATITUDE,LONGITUDE,ELEVATION
0,Valley Head,"(POLYGON ((-85.630563 34.555064, -85.626993 34...",106899,USW00063862,34.5653,-85.6172,310.9
0,Valley Head,"(POLYGON ((-85.630563 34.555064, -85.626993 34...",81046,USC00018469,34.5686,-85.6064,341.7
1,Sylvania,"(POLYGON ((-85.781384 34.565536, -85.776886 34...",48923,US1ALDK0008,34.5346,-85.7563,396.8
3,Rainsville,"(POLYGON ((-85.895725 34.513797, -85.891408 34...",48926,US1ALDK0012,34.4719,-85.8429,365.8
3,Rainsville,"(POLYGON ((-85.895725 34.513797, -85.891408 34...",80992,USC00016762,34.5000,-85.8667,402.9


In [270]:
stations_and_cities.count()

NAME           26205
geometry       26205
index_right    26205
Station        26205
LATITUDE       26205
LONGITUDE      26205
ELEVATION      26205
dtype: int64

In [241]:
stations_and_cities.to_csv(r'stations_withGeo.csv',index=False)